In [1]:
import nltk
import os
import csv
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

In [2]:
train_sents = list(nltk.corpus.conll2002.iob_sents('ned.train'))
dev_sents = list(nltk.corpus.conll2002.iob_sents('ned.testa'))
test_sents = list(nltk.corpus.conll2002.iob_sents('ned.testb'))

## Importing dataset as txt file

In [3]:
with open('/Users/ellyzamaripapas/Code/NER_Project/data/annotated_data/yes-bisaya-1.txt') as file:
    lines = [line.rstrip() for line in file]

In [4]:
lines

['-DOCSTART- -X- O',
 "Aduna'y -X- _ O",
 'pagsaka -X- _ O',
 'sa -X- _ O',
 'presyo -X- _ O',
 'sa -X- _ O',
 'lana -X- _ O',
 ', -X- _ O',
 'epektibo -X- _ O',
 'sa -X- _ O',
 'Martes -X- _ O',
 ', -X- _ O',
 'Hunyo -X- _ O',
 '27 -X- _ O',
 ', -X- _ O',
 '2023 -X- _ O',
 '. -X- _ O',
 '',
 'MOTORISTA -X- _ O',
 ', -X- _ O',
 'PATAY -X- _ O',
 'HUMAN -X- _ O',
 'MALIGSAN -X- _ O',
 'ANG -X- _ O',
 'ULO -X- _ O',
 'SA -X- _ O',
 '10-WHEELER -X- _ O',
 'DUMP -X- _ O',
 'TRUCK -X- _ O',
 'Patay -X- _ O',
 'ang -X- _ O',
 'usa -X- _ O',
 'ka -X- _ O',
 'motorista -X- _ O',
 'sa -X- _ O',
 'usa -X- _ O',
 'ka -X- _ O',
 'vehicular -X- _ O',
 'accident -X- _ O',
 'ang -X- _ O',
 'nahitabo -X- _ O',
 'sa -X- _ O',
 'Barangay -X- _ B-LOC',
 'Caidiocan -X- _ I-LOC',
 'sa -X- _ O',
 'Valencia -X- _ B-LOC',
 'mga -X- _ O',
 'alas-6:20 -X- _ O',
 'sa -X- _ O',
 'gabii -X- _ O',
 'niadtong -X- _ O',
 'Sabado -X- _ O',
 ', -X- _ O',
 'Hunyo -X- _ O',
 '24,2023. -X- _ O',
 'Giila -X- _ O',
 'ang -X

## Converting to list of list per article

In [5]:
main_lst=[]
temp_lst = []
no=1
for line in lines:
    if line=='':
        main_lst.append(temp_lst)
        temp_lst=[]
        print(f'Article: {no}')
        no = no+1
    else:
        temp_lst.append(line)

Article: 1
Article: 2
Article: 3
Article: 4
Article: 5
Article: 6
Article: 7
Article: 8
Article: 9
Article: 10
Article: 11
Article: 12
Article: 13
Article: 14
Article: 15
Article: 16
Article: 17
Article: 18
Article: 19
Article: 20
Article: 21
Article: 22
Article: 23
Article: 24
Article: 25
Article: 26
Article: 27
Article: 28
Article: 29
Article: 30
Article: 31
Article: 32
Article: 33
Article: 34
Article: 35
Article: 36
Article: 37
Article: 38
Article: 39
Article: 40
Article: 41
Article: 42
Article: 43
Article: 44
Article: 45
Article: 46
Article: 47
Article: 48
Article: 49
Article: 50
Article: 51
Article: 52
Article: 53
Article: 54
Article: 55
Article: 56
Article: 57
Article: 58
Article: 59
Article: 60
Article: 61
Article: 62
Article: 63
Article: 64
Article: 65
Article: 66
Article: 67
Article: 68
Article: 69
Article: 70
Article: 71
Article: 72
Article: 73
Article: 74
Article: 75
Article: 76
Article: 77
Article: 78
Article: 79
Article: 80
Article: 81
Article: 82
Article: 83
Article: 84
A

In [6]:
main_lst[1]

['MOTORISTA -X- _ O',
 ', -X- _ O',
 'PATAY -X- _ O',
 'HUMAN -X- _ O',
 'MALIGSAN -X- _ O',
 'ANG -X- _ O',
 'ULO -X- _ O',
 'SA -X- _ O',
 '10-WHEELER -X- _ O',
 'DUMP -X- _ O',
 'TRUCK -X- _ O',
 'Patay -X- _ O',
 'ang -X- _ O',
 'usa -X- _ O',
 'ka -X- _ O',
 'motorista -X- _ O',
 'sa -X- _ O',
 'usa -X- _ O',
 'ka -X- _ O',
 'vehicular -X- _ O',
 'accident -X- _ O',
 'ang -X- _ O',
 'nahitabo -X- _ O',
 'sa -X- _ O',
 'Barangay -X- _ B-LOC',
 'Caidiocan -X- _ I-LOC',
 'sa -X- _ O',
 'Valencia -X- _ B-LOC',
 'mga -X- _ O',
 'alas-6:20 -X- _ O',
 'sa -X- _ O',
 'gabii -X- _ O',
 'niadtong -X- _ O',
 'Sabado -X- _ O',
 ', -X- _ O',
 'Hunyo -X- _ O',
 '24,2023. -X- _ O',
 'Giila -X- _ O',
 'ang -X- _ O',
 'biktima -X- _ O',
 'nga -X- _ O',
 'si -X- _ O',
 'Ruperto -X- _ B-PER',
 'Solamillo -X- _ I-PER',
 'Ausejo -X- _ I-PER',
 'Jr. -X- _ I-PER',
 ', -X- _ O',
 '46 -X- _ O',
 'anyos -X- _ O',
 ', -X- _ O',
 'ulitawo -X- _ O',
 ', -X- _ O',
 'ug -X- _ O',
 'lumolupyo -X- _ O',
 'sa -X- 

In [7]:
#testing for single line
import re
#algo for converting to tuple
tuple(filter(None,[re.sub('[\s_]','',x) for x in main_lst[0][1].split('-')]))

("Aduna'y", 'X', 'O')

## Converting to list of tuple per article

In [8]:
final_train=[]
for article in main_lst:
    temp_lst=[]
    for item in article:
        temp_lst.append(tuple(filter(None,[re.sub('[\s_]','',x) for x in item.split(' ')])))
    final_train.append(temp_lst)

In [9]:
len(final_train)

1158

In [10]:
#final output = final_train
final_train[5]

[('STATE', '-X-', 'O'),
 ('OF', '-X-', 'O'),
 ('CALAMITY', '-X-', 'O'),
 ('SA', '-X-', 'O'),
 ('TIBUOK', '-X-', 'O'),
 ('PILIPINAS', '-X-', 'B-LOC'),
 ('TUNGOD', '-X-', 'O'),
 ('SA', '-X-', 'O'),
 ('ASF', '-X-', 'B-OTHER'),
 (',', '-X-', 'O'),
 ('GITUN-AN', '-X-', 'O'),
 ('Gitun-an', '-X-', 'O'),
 ('karon', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('Department', '-X-', 'B-ORG'),
 ('of', '-X-', 'I-ORG'),
 ('Agriculture', '-X-', 'I-ORG'),
 ('(', '-X-', 'I-ORG'),
 ('DA', '-X-', 'I-ORG'),
 (')', '-X-', 'I-ORG'),
 ('ang', '-X-', 'O'),
 ('pagsugyot', '-X-', 'O'),
 ('ngadto', '-X-', 'O'),
 ('ni', '-X-', 'O'),
 ('Pres.', '-X-', 'O'),
 ('Ferdinand', '-X-', 'B-PER'),
 ('Marcos', '-X-', 'I-PER'),
 ('Jr.', '-X-', 'I-PER'),
 ('nga', '-X-', 'O'),
 ('magdeklarar', '-X-', 'O'),
 ('og', '-X-', 'O'),
 ('state', '-X-', 'O'),
 ('of', '-X-', 'O'),
 ('calamity', '-X-', 'O'),
 ('taliwala', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('pagkaylap', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('African', '-X-', 'B-OTHER'),
 ('Swine', 

In [11]:
#splitting
training_data = final_train[0:811]
testing_data = final_train[811:1043]
dev_data = final_train[1043:]

## Feature Extraction

In [12]:
def read_clusters(cluster_file):
    word2cluster = {}
    with open(cluster_file) as i:
        for line in i:
            word, cluster = line.strip().split('\t')
            word2cluster[word] = cluster
    return word2cluster


def word2features(sent, i, word2cluster):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.cluster=%s' % word2cluster[word.lower()] if word.lower() in word2cluster else "0",
        'postag=' + postag
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS')

    if i > 1: 
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2
        ])        

        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.extend([
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2
        ])

        
    return features


def sent2features(sent, word2cluster):
    return [word2features(sent, i, word2cluster) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters("/Users/ellyzamaripapas/Code/NER_Project/data/cebtenten.tsv")

In [13]:
sent2features(training_data[0], word2cluster)[0]

['bias',
 'word.lower=-docstart-',
 'word[-3:]=RT-',
 'word[-2:]=T-',
 'word.isupper=True',
 'word.istitle=False',
 'word.isdigit=False',
 '0',
 'postag=-X-',
 'BOS',
 "+1:word.lower=aduna'y",
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=-X-',
 '+2:word.lower=pagsaka',
 '+2:word.istitle=False',
 '+2:word.isupper=False',
 '+2:postag=-X-']

In [14]:
X_train = [sent2features(s, word2cluster) for s in training_data]
y_train = [sent2labels(s) for s in training_data]

X_dev = [sent2features(s, word2cluster) for s in dev_data]
y_dev = [sent2labels(s) for s in dev_data]

X_test = [sent2features(s, word2cluster) for s in testing_data]
y_test = [sent2labels(s) for s in testing_data]

## Training

In [15]:
crf = crfsuite.CRF(
    verbose='true',
    algorithm='lbfgs',
    max_iterations=100
)

try:
    crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
except AttributeError:
    pass
predictions = crf.predict(X_test)

loading training data to CRFsuite: 100%|██████████| 811/811 [00:00<00:00, 921.21it/s] 


loading dev data to CRFsuite: 100%|██████████| 115/115 [00:00<00:00, 1657.44it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 80780
Seconds required: 0.434

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.11  loss=60407.93 active=80780 precision=0.093  recall=0.111  F1=0.102  Acc(item/seq)=0.841 0.183  feature_norm=1.00
Iter 2   time=0.06  loss=58685.61 active=80780 precision=0.093  recall=0.111  F1=0.102  Acc(item/seq)=0.841 0.183  feature_norm=0.94
Iter 3   time=0.06  loss=55806.93 active=80780 precision=0.093  recall=0.111  F1=0.102  Acc(item/seq)=0.841 0.183  feature_norm=0.87
Iter 4   time=0.06  loss=53543.26 active=80780 precision=0.093  recall=0.111  F1=0.102  Acc(item/seq)=0.841 0.183  feature_norm=0.96
Iter 5   time=0.06  loss=42125.60 active=80780 pr

In [16]:
import joblib
import os

OUTPUT_PATH = "crf-model"
OUTPUT_FILE = "crf-model-yes-bisaya1"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

['crf-model/crf-model-yes-bisaya1']

## Evaluation

In [17]:
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

example_sent = training_data[3]

print("Sentence:", ' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent, word2cluster)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Sentence: LALAKI SA INDIA , NIPUYO OG DUL-AN 2 KA TUIG SA USA KA 5-STAR HOTEL NGA WALA’Y BAYAD-BAYAD Usa ka lalaki ang giimbestigaran sa kapulisan sa India human giingong nagpuyo kini og halos duha ka tuig sa usa ka five-star hotel sa New Delhi nga wala'y bayad-bayad. Nag-book og kwarto si Ankush Dutta sa Roseate House hotel niadtong May 30 , 2019 ug mo-check out unta dayon sa sunod adlaw. Apan na-extend kini og 603 ka mga adlaw hangtod sa iyang pagbiya niadtong Jan. 22 , 2021 , ug gibiyaan ang iyang $ 70,000 nga balayran. Nipasaka og reklamo sa kapulisan ang hotel managers kontra sa pipila ka mga empleyado niini tungod sa " conspiracy , forgery and cheating. " Dugang pa , giingong gisuholan ni Dutta ang pipila ka mga kawani sa hotel aron pagmaniobra sa in-house software systems .
Predicted: O O B-LOC O O O O O O O O O O O O O O O O O O O O O O O B-LOC O O O O O O O O O O O O O O O B-LOC I-LOC O O O O O O O B-PER I-PER O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O O O O O O O O

In [18]:
labels = list(crf.classes_)
labels.remove("O")
y_pred = crf.predict(X_test)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
from sklearn import metrics

# Flatten the y_test and y_pred sequences
y_test_flat = [item for sublist in y_test for item in sublist]
y_pred_flat = [item for sublist in y_pred for item in sublist]

# Print the classification report
print(metrics.classification_report(y_test_flat, y_pred_flat))

#print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels))

              precision    recall  f1-score   support

       B-LOC       0.63      0.62      0.63       180
       B-ORG       0.77      0.58      0.66       183
     B-OTHER       0.69      0.47      0.56       163
       B-PER       0.87      0.75      0.81       370
       I-LOC       0.70      0.63      0.66       205
       I-ORG       0.84      0.82      0.83       568
     I-OTHER       0.71      0.64      0.67       271
       I-PER       0.85      0.80      0.83       291
           O       0.97      0.99      0.98     10999

    accuracy                           0.94     13230
   macro avg       0.78      0.70      0.74     13230
weighted avg       0.94      0.94      0.94     13230



In [19]:
import sklearn_crfsuite
import numpy as np

def display_crf_weights(crf, top=30):
    # Get the feature weights from the CRF model
    feature_weights = crf.state_features_

    # Extract the values from the dictionary
    weights = np.array(list(feature_weights.values()))

    # Sort the feature weights by absolute value
    sorted_indices = np.argsort(np.abs(weights))[::-1]

    # Select the top feature weights
    top_indices = sorted_indices[:top]

    # Display the top feature weights and their corresponding features
    for idx in top_indices:
        feature = list(feature_weights.keys())[idx]
        weight = weights[idx]
        print(f"Feature: {feature}, Weight: {weight}")

# Assuming 'crf' is your CRF model object
display_crf_weights(crf, top=30)

Feature: ('word[-2:]=00', 'O'), Weight: 3.177969
Feature: ('-1:word.lower=si', 'B-PER'), Weight: 3.120408
Feature: ('-1:word.lower=#', 'O'), Weight: -2.826866
Feature: ('0', 'O'), Weight: -2.791014
Feature: ('-1:word.lower=ni', 'B-PER'), Weight: 2.620423
Feature: ('-2:word.lower=si', 'B-PER'), Weight: 2.53146
Feature: ('-2:word.lower=ni', 'B-PER'), Weight: 2.40477
Feature: ('word.lower=sa', 'O'), Weight: 2.40277
Feature: ('word.istitle=False', 'O'), Weight: 2.304377
Feature: ('-1:word.lower=sa', 'B-LOC'), Weight: 2.301012
Feature: ('word.isupper=False', 'O'), Weight: 2.296928
Feature: ('-1:word.lower=niadtong', 'O'), Weight: 2.258597
Feature: ('word[-3:]=000', 'O'), Weight: 2.254797
Feature: ('-1:word.lower=#', 'B-OTHER'), Weight: 2.16825
Feature: ('word[-2:]=ia', 'O'), Weight: -2.153007
Feature: ('word.lower=deped', 'B-ORG'), Weight: 2.116828
Feature: ('+1:word.lower=sab', 'O'), Weight: 2.067877
Feature: ('word[-2:]=Ed', 'B-ORG'), Weight: 1.956704
Feature: ('EOS', 'O'), Weight: 1.8865

## Finding the optimal hyperparameters

In [20]:
import scipy
import sklearn_crfsuite
from sklearn.metrics import make_scorer, classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Create a custom CRFWrapper class to add the keep_tempfiles attribute
class CRFWrapper(sklearn_crfsuite.CRF):
    def __init__(self, keep_tempfiles=True, c1=0.1, c2=0.1, **kwargs):
        super().__init__(c1=c1, c2=c2, **kwargs)
        self.keep_tempfiles = keep_tempfiles

# Instantiate the custom CRFWrapper class with keep_tempfiles=True and default c1, c2 values
crf = CRFWrapper(
    keep_tempfiles=True,
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# Define f1_scorer
f1_scorer = make_scorer(sklearn_crfsuite.metrics.flat_f1_score,
                        average='weighted', labels=crf.classes_)


# Define RandomizedSearchCV
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
# Fit the RandomizedSearchCV
rs.fit(X_train, y_train)

# Get the best estimator from RandomizedSearchCV
best_crf = rs.best_estimator_

# Predictions using the best estimator
predictions = best_crf.predict(X_test)

# Print the classification report
print(classification_report(y_test_flat, y_pred_flat))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
